PARTE DE ENTRENAMIENTO DE LAS REDES

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from PIL import Image
import os

In [ ]:
def prepare_image_to_segmentation(path,size=(160,160)):
    #devuelve tensor de tamaño (1,160,160,3) para meter en la red
    return np.array(load_img(path, target_size=size,color_mode="rgb")).reshape((1,)+size+(3,))

In [ ]:
from keras_unet_collection import models,losses

#model_Unet = models.unet_2d((256,256,3),filter_num=[64,128,256,512,1024],n_labels=2,stack_num_down=2,stack_num_up=2,activation='ReLU',output_activation='Sigmoid',batch_norm=True,pool=False,unpool=False,backbone='VGG16',weights='imagenet',freeze_backbone=True,freeze_batch_norm=True,name='unetlibreria')
from keras_unet_collection import models,losses

model_Unet = models.unet_2d((None,None,3),filter_num=[64,128,256,512,1024],n_labels=2,stack_num_down=2,stack_num_up=2,activation='ReLU',output_activation='Sigmoid',batch_norm=True,pool=False,unpool=False,weights='imagenet',freeze_backbone=False,freeze_batch_norm=False,name='unetlibreria')

In [ ]:
import numpy as np
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
from PIL import ImageOps, Image
def obtain_mask(i):
    """Quick utility to display a model's prediction."""
    mask = np.argmax(i, axis=-1)
    mask = np.expand_dims(mask,axis=-1)
    return mask

In [ ]:
import numpy as np
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
from PIL import ImageOps, Image

def display_mask(i):
    """Quick utility to display a model's prediction."""
    
    mask = np.argmax(i, axis=-1)
    mask = np.expand_dims(mask,axis=-1)
    img = ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask))
    display(img)

In [ ]:
model_Unet.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=1e-3),metrics=['accuracy',tf.keras.metrics.IoU(num_classes=2, target_class_ids=[0,1])])

In [ ]:
def unet(img_size,num_classes):
    
    
    #Input
    layer = keras.Input(shape=img_size)
    input_layer = layer 
    
    output_block_layer = []
    #Bloque de codificador
    for i in [64,128,256,512]:
        layer = layers.Conv2D(i,3,strides=1,padding="same",activation="relu")(layer)
        layer = layers.BatchNormalization()(layer)
        layer = layers.Conv2D(i,3,strides=1,padding="same",activation="relu")(layer)
        layer = layers.BatchNormalization()(layer)
        
        #Guardo la capa para la parte de decodificacion
        output_block_layer.append(layer)
        
        layer = layers.MaxPool2D(2)(layer)
        layer = layers.Dropout(0.2)(layer)
    
    #Cuello de botella con 1024 filtros
    layer = layers.Conv2D(1024,3,strides=1,padding="same",activation="relu")(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Conv2D(1024,3,strides=1,padding="same",activation="relu")(layer)
    layer = layers.BatchNormalization()(layer)
    
    indice = 1
    #Bloque decodificador
    for i in [512,256,128,64]:
        # upsample
        layer = layers.Conv2DTranspose(i, 3, 2, padding="same")(layer)
        # concatenate
        layer = layers.concatenate([layer, output_block_layer[len(output_block_layer)-indice]])
        indice = indice+1
        # dropout
        layer = layers.Dropout(0.2)(layer)
        # Conv2D twice with ReLU activation
        layer = layers.Conv2D(i, 3, padding = "same", activation = "relu")(layer)
        # Conv2D then ReLU activation
        layer = layers.Conv2D(i, 3, padding = "same", activation = "relu")(layer)
        
    output = layers.Conv2D(num_classes,3, padding="same", activation = "softmax")(layer)
    
    return tf.keras.Model(input_layer,output,name="U-Net")

In [ ]:
import os
import shutil

input_dir = "./TEM-Nano-Particle-Cell-Dataset-master/TEM-Nano-Particle-Cell-Dataset-master"
target_dir = "./TEM-Nano-Particle-Cell-Dataset-master/TEM-Nano-Particle-Cell-Dataset-master/Validation"
img_size = (256,256)
num_classes = 2
batch_size = 32

print(os.listdir(input_dir))

print(os.listdir(target_dir))

input_img_paths = [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".tif")
    ]
target_img_paths = [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".tif") and not fname.startswith(".")
    ]
#Para corregir donde se carga la primera imagen
target_img_paths.insert(0,target_img_paths.pop(8))

print("Number of samples:", len(input_img_paths))

for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)

In [ ]:


# Display input image #7
im = load_img(input_img_paths[0])
display(im)
# Display auto-contrast version of corresponding target (per-pixel categories)
img = load_img(target_img_paths[0])
display(img)



In [ ]:
#Prueba hecha en colab
from PIL import Image
def read_tif_img(path):
    img = Image.open(path)
    print(path)
    img = img.resize((160,160))
    img_rgb = img.convert('RGB')
    imagen = np.array(img_rgb)
    return imagen

#pred = model.predict(imagen)

#mask = np.argmax(pred[0,...,1:3], axis=-1)
#mask = np.expand_dims(mask, axis=-1)
#img = ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask))
#display(img)

In [ ]:
def mask_preprocessing(tensor):
    mask = np.argmax(tensor, axis=-1)
    mask = np.expand_dims(mask,axis=-1)
    return mask

In [ ]:

size = (256,256)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=45,     #Random rotation between 0 and 45
        width_shift_range=0.2,   #% shift
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='wrap', cval=125)    #Also try nearest, constant, reflect, wrap

maskgen = ImageDataGenerator(
        rotation_range=45,     #Random rotation between 0 and 45
        width_shift_range=0.2,   #% shift
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='wrap', cval=125)    #Also try nearest, constant, reflect, wrap




In [ ]:
import PIL
from PIL import Image
import os

def convert_to_grayscale_saved_augmentated_masks(directory):
	for picture in os.listdir(directory):
		load_mask = np.array(load_img(directory + picture,color_mode="rgb"))
		mask = np.argmax(load_mask, axis=-1)
		mask = np.expand_dims(mask,axis=-1)
		img = ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask))
		img.save(directory+picture.split('.')[0]+'.tif')
		os.remove(directory+picture)

def change_format_augmentated_images(directory):
	for picture in os.listdir(directory):
		img = np.array(load_img(directory + picture,color_mode="rgb"))
		img = ImageOps.autocontrast(keras.preprocessing.image.array_to_img(img))
		img.save(directory+picture.split('.')[0]+'.tif')
		os.remove(directory+picture)


In [ ]:
all_images = []

all_masks = []

for i in range(len(input_img_paths)):
    all_images.append(np.array(load_img(input_img_paths[i], target_size=size,color_mode="rgb")))
    all_masks.append(np.array(load_img(target_img_paths[i], target_size=size,color_mode="grayscale"))//255)

all_images = np.array(all_images)
all_masks = np.array(all_masks)
all_masks_background_tags = (1-np.array(all_masks)).reshape((17,)+size)
all_masks = np.stack((all_masks_background_tags,all_masks,np.zeros((17,)+size)),axis = -1)
all_masks.shape


In [ ]:
datagen.fit(all_images,augment=True,seed=109)
maskgen.fit(all_masks,augment=True,seed=109)
imagen_datagen = datagen.flow(all_images,seed=109,save_to_dir='./data/augmentation/images/')
mask_datagen = maskgen.flow(all_masks,seed=109,save_to_dir='./data/augmentation/masks/')

ANTES DE EJECUTAR LA SIGUIENTE CELDA VACIE LAS CARPETAS DE LAS FUNCIONES

In [ ]:
num_augmentation = 20

for i in range(num_augmentation):
    imagen_datagen.next()
    mask_datagen.next()



convert_to_grayscale_saved_augmentated_masks('data/augmentation/masks/')
change_format_augmentated_images('data/augmentation/images/')   

In [ ]:
augmentatde_img_dir = 'data/augmentation/images/'
augmentatde_maske_dir = 'data/augmentation/masks/'

input_aug_img_paths = sorted([
        os.path.join(augmentatde_img_dir, fname)
        for fname in os.listdir(augmentatde_img_dir)
        if fname.endswith(".tif")
    ])
target_aug_img_paths = sorted([
        os.path.join(augmentatde_maske_dir, fname)
        for fname in os.listdir(augmentatde_maske_dir)
        if fname.endswith(".tif") and not fname.startswith(".")
    ])

In [ ]:
len(target_aug_img_paths)

In [ ]:
import random
# Split our img paths into a training and a validation set
val_samples = 50
test_samples = 20
random.Random(1337).shuffle(input_aug_img_paths)
random.Random(1337).shuffle(target_aug_img_paths)

train_input_img_paths = input_aug_img_paths[:-(val_samples + test_samples)]
train_target_img_paths = target_aug_img_paths[:-(val_samples + test_samples)]
val_input_img_paths = input_aug_img_paths[-(val_samples + test_samples):-test_samples]
val_target_img_paths = target_aug_img_paths[-(val_samples + test_samples):-test_samples]
test_input_img_path = input_aug_img_paths[-test_samples:]
test_target_img_paths = target_aug_img_paths[-test_samples:]

total_train_images = len(train_input_img_paths)
print(len(train_input_img_paths),len(val_input_img_paths),len(test_input_img_path))
# Instantiate data Sequences for each split
train_x = []
train_y = []
for i in range(len(train_input_img_paths)):
    train_x.append(np.array(load_img(train_input_img_paths[i], target_size=size,color_mode="rgb")))
    train_y.append(np.array(load_img(train_target_img_paths[i], target_size=size,color_mode="grayscale"))//255)


val_x = []
val_y = []
for i in range(len(val_input_img_paths)):
    val_x.append(np.array(load_img(val_input_img_paths[i], target_size=size,color_mode="rgb")))
    val_y.append(np.array(load_img(val_target_img_paths[i], target_size=size,color_mode="grayscale"))//255)

test_x = []
test_y = []
for i in range(len(test_input_img_path)):
    test_x.append(np.array(load_img(test_input_img_path[i], target_size=size,color_mode="rgb")))
    test_y.append(np.array(load_img(test_target_img_paths[i], target_size=size,color_mode="grayscale"))//255)

train_x = np.array(train_x)
#train_x = np.stack((train_x,train_x,train_x),axis=-1)
train_y = np.array(train_y).reshape((total_train_images,)+size)

train_y_0 = (1-np.array(train_y)).reshape((total_train_images,)+size)
#train_y_0 = (1-np.array(train_y)).reshape(32,160,160)
#train_y = np.stack((train_y_0,train_y,np.zeros((total_train_images,)+size)),axis=-1).astype("uint8")
train_y = np.stack((train_y_0,train_y),axis=-1).astype("uint8")

test_x = np.array(test_x).reshape((test_samples,)+size+(3,))
#test_x = np.stack((test_x,test_x,test_x),axis=-1)


test_y_0 = (1-np.array(test_y)).reshape((test_samples,)+size)
test_y = np.array(test_y).reshape((test_samples,)+size)

test_y = np.stack((test_y_0,test_y),axis=-1).astype("uint8")
#test_y = np.stack((test_y_0,test_y,np.zeros((val_samples,)+size)),axis=-1).astype("uint8")


val_x = np.array(val_x).reshape((val_samples,)+size+(3,))
#test_x = np.stack((test_x,test_x,test_x),axis=-1)


val_y_0 = (1-np.array(val_y)).reshape((val_samples,)+size)
val_y = np.array(val_y).reshape((val_samples,)+size)

val_y = np.stack((val_y_0,val_y),axis=-1).astype("uint8")
#val_y = np.stack((val_y_0,val_y,np.zeros((val_samples,)+size)),axis=-1).astype("uint8")
train_y.shape

In [ ]:
val_y.shape

In [ ]:
#Entreno modelo libreria
callbacks = [
    keras.callbacks.ModelCheckpoint("entrenadoTL.h5", save_best_only=True)
]

epochs = 25

model_Unet.fit(train_x,train_y,batch_size=16,validation_data=(val_x,val_y),epochs=epochs,verbose=1,callbacks=callbacks)

In [ ]:
ouputs = model_Unet.predict(test_x)
ouputs.shape

In [ ]:
outputs = model_Unet.predict(test_x)

In [ ]:

modelo_cargado_oxford = keras.models.load_model("oxford_segmentation7032023.h5")


In [ ]:
print((input_aug_img_paths))
print((target_aug_img_paths))

In [ ]:
import random
# Preparación de datos
# Split our img paths into a training and a validation set
val_samples = 50
test_samples = 20
random.Random(1337).shuffle(input_aug_img_paths)
random.Random(1337).shuffle(target_aug_img_paths)

train_input_img_paths = input_aug_img_paths[:-(val_samples + test_samples)]
train_target_img_paths = target_aug_img_paths[:-(val_samples + test_samples)]
val_input_img_paths = input_aug_img_paths[-(val_samples + test_samples):-test_samples]
val_target_img_paths = target_aug_img_paths[-(val_samples + test_samples):-test_samples]
test_input_img_path = input_aug_img_paths[-test_samples:]
test_target_img_paths = target_aug_img_paths[-test_samples:]

print(test_input_img_path)

total_train_images = len(train_input_img_paths)
print(len(train_input_img_paths),len(val_input_img_paths),len(test_input_img_path))
# Instantiate data Sequences for each split
train_x = []
train_y = []
for i in range(len(train_input_img_paths)):
    train_x.append(np.array(load_img(train_input_img_paths[i], target_size=size,color_mode="rgb")))
    train_y.append(np.array(load_img(train_target_img_paths[i], target_size=size,color_mode="grayscale"))//255)


val_x = []
val_y = []
for i in range(len(val_input_img_paths)):
    val_x.append(np.array(load_img(val_input_img_paths[i], target_size=size,color_mode="rgb")))
    val_y.append(np.array(load_img(val_target_img_paths[i], target_size=size,color_mode="grayscale"))//255)

test_x = []
test_y = []
for i in range(len(test_input_img_path)):
    test_x.append(np.array(load_img(test_input_img_path[i], target_size=size,color_mode="rgb")))
    test_y.append(np.array(load_img(test_target_img_paths[i], target_size=size,color_mode="grayscale"))//255)

train_x = np.array(train_x)
#train_x = np.stack((train_x,train_x,train_x),axis=-1)
#train_y = np.array(train_y).reshape(16,256,256)
train_y = np.array(train_y).reshape((total_train_images,)+size)

train_y_0 = (1-np.array(train_y)).reshape((total_train_images,)+size)
#train_y_0 = (1-np.array(train_y)).reshape(32,160,160)
train_y = np.stack((train_y_0,train_y,np.zeros((total_train_images,)+size)),axis=-1).astype("uint8")
#train_y = np.stack((train_y_0,train_y),axis=-1).astype("uint8")

test_x = np.array(test_x).reshape((test_samples,)+size+(3,))
#test_x = np.stack((test_x,test_x,test_x),axis=-1)


test_y_0 = (1-np.array(test_y)).reshape((test_samples,)+size)
test_y = np.array(test_y).reshape((test_samples,)+size)

#test_y = np.stack((test_y_0,test_y),axis=-1).astype("uint8")
test_y = np.stack((test_y_0,test_y,np.zeros((test_samples,)+size)),axis=-1).astype("uint8")


val_x = np.array(val_x).reshape((val_samples,)+size+(3,))
#test_x = np.stack((test_x,test_x,test_x),axis=-1)


val_y_0 = (1-np.array(val_y)).reshape((val_samples,)+size)
val_y = np.array(val_y).reshape((val_samples,)+size)

#val_y = np.stack((val_y_0,val_y),axis=-1).astype("uint8")
val_y = np.stack((val_y_0,val_y,np.zeros((val_samples,)+size)),axis=-1).astype("uint8")
train_y.shape

In [ ]:
#Entreno modelo oxford
callbacks = [
    keras.callbacks.ModelCheckpoint("modeloOxford.h5", save_best_only=True)
]
modelo_cargado_oxford.compile(optimizer="adam", 
                       loss=tf.keras.losses.categorical_crossentropy,#binary_focal_crossentropy
                       metrics=["accuracy",tf.keras.metrics.IoU(num_classes=2, target_class_ids=[1])])
epochs = 25

modelo_cargado_oxford.fit(train_x,train_y,batch_size=8,validation_data=(val_x,val_y),epochs=epochs,verbose=1,callbacks=callbacks)

In [ ]:
out = display_mask(modelo_cargado_oxford.predict(test_x)[5])

In [ ]:
plt.imshow(test_x[3])
plt.show()

In [ ]:
#Entreno modelo propio entrenado con dataset PETS

modelo_cargado =  keras.models.load_model("UNET07032023.h5")

callbacks = [
    keras.callbacks.ModelCheckpoint("modeloPropio.h5", save_best_only=True)
]
modelo_cargado.compile(optimizer="adam", 
                       loss=tf.keras.losses.categorical_crossentropy,#binary_focal_crossentropy
                       metrics=["accuracy",tf.keras.metrics.IoU(num_classes=2, target_class_ids=[1])])
epochs = 25

modelo_cargado.fit(train_x,train_y,batch_size=16,validation_data=(val_x,val_y),epochs=epochs,verbose=1,callbacks=callbacks)